In [1]:
%store -r df_listings

In [2]:
df_listings.shape

(13945, 79)

In [3]:
%store -r df_calendar_cleaned

In [4]:
%store -r merged_neighborhoods

In [5]:
df_listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
df_listings['price'] = df_listings['price'].astype(str).str.replace('[\$,]', '', regex=True)

In [8]:
import pandas as pd
df_listings['price'] = pd.to_numeric(df_listings['price'], errors='coerce')

In [9]:
def integrate_calendar_prices(df_listings, df_calendar_cleaned):
    """
    Fill missing listing prices with calendar median prices
    """
    print(f"Listings before: {len(df_listings)} ({df_listings['price'].notna().sum()} with prices)")
    
    # Get median price per listing from calendar
    calendar_prices = (df_calendar_cleaned
                      .groupby('listing_id')['price']
                      .median()
                      .reset_index()
                      .rename(columns={'price': 'calendar_price'}))
    
    # Merge and fill missing prices
    df_merged = df_listings.merge(calendar_prices, left_on='id', right_on='listing_id', how='left')
    df_merged['price'] = df_merged['price'].fillna(df_merged['calendar_price'])
    
    # Clean and filter
    df_final = df_merged.dropna(subset=['price'])
    df_final = df_final[(df_final['price'] > 0) & (df_final['price'] <= 10000)]
    
    print(f"Listings after: {len(df_final)} (recovered {len(df_final) - df_listings['price'].notna().sum()})")
    print(f"Price range: ${df_final['price'].min():.0f} - ${df_final['price'].max():.0f}")
    
    return df_final.drop(columns=['listing_id', 'calendar_price'])

In [10]:
df_listings = integrate_calendar_prices(df_listings, df_calendar_cleaned)

Listings before: 13945 (8898 with prices)
Listings after: 13934 (recovered 5036)
Price range: $8 - $10000


In [11]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13934 entries, 0 to 13944
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            13934 non-null  int64  
 1   listing_url                                   13934 non-null  object 
 2   scrape_id                                     13934 non-null  int64  
 3   last_scraped                                  13934 non-null  object 
 4   source                                        13934 non-null  object 
 5   name                                          13934 non-null  object 
 6   description                                   13340 non-null  object 
 7   neighborhood_overview                         6610 non-null   object 
 8   picture_url                                   13934 non-null  object 
 9   host_id                                       13934 non-null  int6

In [12]:
listings = df_listings.copy()

In [13]:
bool_cols = [
    "host_is_superhost", "instant_bookable", "has_availability",
    "host_has_profile_pic", "host_identity_verified"
]
for col in bool_cols:
    listings[col] = (
        listings[col]
        .map({"t": True, "f": False})          
        .fillna(False)                      
        .astype("bool")
    )

In [14]:
date_cols = ["host_since", "first_review", "last_review"]
listings[date_cols] = listings[date_cols].apply(
    pd.to_datetime, errors="coerce"
)

In [15]:
num_cols = [
    "accommodates", "bedrooms", "beds", "bathrooms",
    "minimum_nights", "maximum_nights",
    "availability_30","availability_60","availability_90", "availability_365",
    "number_of_reviews", "review_scores_rating"
]
listings[num_cols] = (
    listings[num_cols]
      .apply(pd.to_numeric, errors="coerce")
)

In [16]:
clip_vals = listings[num_cols].quantile(0.99)
listings[num_cols] = listings[num_cols].clip(upper=clip_vals,axis=1)

In [17]:
for c in ["instant_bookable", "has_availability",
          "host_has_profile_pic", "host_identity_verified"]:
    listings[c] = listings[c].map({'t': True, 'f': False}).fillna(False)

# host_acceptance_rate  "%→float0-1"  +  median fill
lst = listings
lst['host_acceptance_rate'] = (
    lst['host_acceptance_rate'].str.rstrip('%').astype(float).div(100)
)
lst['host_acceptance_rate'].fillna(lst['host_acceptance_rate'].median(), inplace=True)

#universal median/mode impute (anything still NaN)
num = lst.select_dtypes('number').columns
obj = lst.select_dtypes('object').columns.difference(['amenities'])
lst[num] = lst[num].fillna(lst[num].median())
lst[obj] = lst[obj].fillna('Unknown')

# 99-th percentile cap for every numeric col (price already safe, this extends to others)
caps = lst[num].quantile(0.99)
lst[num] = lst[num].clip(upper=caps, axis=1)

print("✅ Listings clean: no NaNs, outliers capped →", lst.shape)

✅ Listings clean: no NaNs, outliers capped → (13934, 79)


In [18]:
lst.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13934 entries, 0 to 13944
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            13934 non-null  int64         
 1   listing_url                                   13934 non-null  object        
 2   scrape_id                                     13934 non-null  int64         
 3   last_scraped                                  13934 non-null  object        
 4   source                                        13934 non-null  object        
 5   name                                          13934 non-null  object        
 6   description                                   13934 non-null  object        
 7   neighborhood_overview                         13934 non-null  object        
 8   picture_url                                   13934 non-null  object   

In [21]:
%store lst

Stored 'lst' (DataFrame)


In [ ]:
#host_since, first_review, andb last_review for feature engineering

In [20]:
#host_since, first_review, and last_review for feature engineering 